In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cmx
import sys
import os
sys.path.append(os.path.abspath("../"))

from time import time
from mpl_toolkits.mplot3d import Axes3D
from learn import CompassModel, from_file, angdist

names = [
    # "seville-cr-32-20170321",
    "seville-cr-32-20170621",
    # "seville-cr-32-20170921",
    "seville-cr-32-20171221",
    "seville-cr-32-20170601"
]

# Load training data
x, y = CompassModel.load_dataset(names[:-1], pol=True, y_shape=(-1, 1))
x = x.reshape((x.shape[0], -1))
# Clean training data
i = np.all(~np.isnan(x), axis=1)
x = x.reshape((x.shape[0], -1, 2))
x, y = x[i], y[i].squeeze()
print "Train:", x.shape, y.shape

# Load testing data
x_test, y_test = CompassModel.load_dataset(names[-1:], pol=True, y_shape=(-1, 1))
x_test = x_test.reshape((x_test.shape[0], -1))
# Clean testing data
i = np.all(~np.isnan(x_test), axis=1)
x_test = x_test.reshape((x_test.shape[0], -1, 2))
x_test, y_test = x_test[i], y_test[i].squeeze()
print "Test:", x_test.shape, y_test.shape

Using TensorFlow backend.


Loading 'seville-cr-32-20170621.npz' ...


Loading 'seville-cr-32-20171221.npz' ...


Train: (17280, 6208, 2) (17280,)
Loading 'seville-cr-32-20170601.npz' ...


Test: (10440, 6208, 2) (10440,)


In [3]:
from learn.whitening import transform, pca, zca

x_pca = transform(x, func=pca, reshape='last', save_filepath='pca2.npz')
x_pca_test = transform(x_test, func=pca, reshape='last', load_filepath='pca2.npz')

learn.whitening: DEBUG: x.shape = (107274240, 2), n = 107274240, d = 2


learn.whitening: DEBUG: x.min = -1.57, x.max = 1.57, x.mean = 0.89


learn.whitening: DEBUG: Creating the correlation matrix...


learn.whitening: DEBUG: C.shape = (2, 2)


learn.whitening: DEBUG: Applying SVD...


...

learn.whitening: DEBUG: Applying whitening to x...


learn.whitening: DEBUG: Before normalization: w.sum = 6.27


learn.whitening: DEBUG: w.min = -0.00, w.max = 5.17


learn.whitening: DEBUG: w.row.sum.mean = 3.14


learn.whitening: DEBUG: w.col.sum.mean = 3.14


In [11]:
print x_pca.max(axis=(0, 1)), x_pca.min(axis=(0, 1))

[ 8.12600994  0.73571482]

 [-8.12592429 -0.22937105]


In [12]:
model = from_file("simple-model.yaml")
model.compile(optimizer="rmsprop", loss="mae", metrics=["accuracy"])
model.summary()

hist = model.train((x_pca.reshape((x.shape[0], 1, -1, 2)), y), valid_data=(x_pca_test.reshape((x_test.shape[0], 1, -1, 2)), y_test), epochs=500)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
SkyFeatures (InputLayer)     (None, 1, 6208, 2)        0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 1, 6208, 1)        3         
_________________________________________________________________
Flatten1 (Flatten)           (None, 6208)              0         
_________________________________________________________________
Hidden1 (Dense)              (None, 360)               2235240   
_________________________________________________________________
Compass (Dense)              (None, 1)                 361       
Total params: 2,235,604
Trainable params: 2,235,604
Non-trainable params: 0
_________________________________________________________________
Train on 17280 samples, validate on 10440 samples


Epoch 1/500


  360/17280 [..............................] - ETA: 23s - loss: 1.3528 - acc: 0.0000e+00

  720/17280 [>.............................] - ETA: 19s - loss: 74.6583 - acc: 0.0000e+00

 1080/17280 [>.............................] - ETA: 18s - loss: 66.9025 - acc: 0.0000e+00

...

 9720/17280 [===============>..............] - ETA: 9s - loss: 1.2456 - acc: 0.0000e+00 

10080/17280 [================>.............] - ETA: 9s - loss: 1.2266 - acc: 0.0000e+00

10440/17280 [=================>............] - ETA: 8s - loss: 1.2175 - acc: 0.0000e+00

10800/17280 [=================>............] - ETA: 8s - loss: 1.2683 - acc: 0.0000e+00

11160/17280 [==================>...........] - ETA: 7s - loss: 1.2844 - acc: 0.0000e+00

In [12]:
model = from_file("simple-model.yaml")
model.compile(optimizer="rmsprop", loss="mae", metrics=["accuracy"])
model.summary()

hist = model.train((x_pca.reshape((x.shape[0], 1, -1, 2)), y), valid_data=(x_pca_test.reshape((x_test.shape[0], 1, -1, 2)), y_test), epochs=500)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
SkyFeatures (InputLayer)     (None, 1, 6208, 2)        0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 1, 6208, 1)        3         
_________________________________________________________________
Flatten1 (Flatten)           (None, 6208)              0         
_________________________________________________________________
Hidden1 (Dense)              (None, 360)               2235240   
_________________________________________________________________
Compass (Dense)              (None, 1)                 361       
Total params: 2,235,604
Trainable params: 2,235,604
Non-trainable params: 0
_________________________________________________________________
Train on 17280 samples, validate on 10440 samples


Epoch 1/500


  360/17280 [..............................] - ETA: 23s - loss: 1.3528 - acc: 0.0000e+00

  720/17280 [>.............................] - ETA: 19s - loss: 74.6583 - acc: 0.0000e+00

 1080/17280 [>.............................] - ETA: 18s - loss: 66.9025 - acc: 0.0000e+00

...

 9720/17280 [===============>..............] - ETA: 10s - loss: 1.3071 - acc: 3.0864e-04

10080/17280 [================>.............] - ETA: 10s - loss: 1.2905 - acc: 2.9762e-04

10440/17280 [=================>............] - ETA: 9s - loss: 1.2797 - acc: 2.8736e-04 

10800/17280 [=================>............] - ETA: 9s - loss: 1.3357 - acc: 2.7778e-04

11160/17280 [==================>...........] - ETA: 9s - loss: 1.3739 - acc: 2.6882e-04

In [12]:
model = from_file("simple-model.yaml")
model.compile(optimizer="rmsprop", loss="mae", metrics=["accuracy"])
model.summary()

hist = model.train((x_pca.reshape((x.shape[0], 1, -1, 2)), y), valid_data=(x_pca_test.reshape((x_test.shape[0], 1, -1, 2)), y_test), epochs=500)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
SkyFeatures (InputLayer)     (None, 1, 6208, 2)        0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 1, 6208, 1)        3         
_________________________________________________________________
Flatten1 (Flatten)           (None, 6208)              0         
_________________________________________________________________
Hidden1 (Dense)              (None, 360)               2235240   
_________________________________________________________________
Compass (Dense)              (None, 1)                 361       
Total params: 2,235,604
Trainable params: 2,235,604
Non-trainable params: 0
_________________________________________________________________
Train on 17280 samples, validate on 10440 samples


Epoch 1/500


  360/17280 [..............................] - ETA: 23s - loss: 1.3528 - acc: 0.0000e+00

  720/17280 [>.............................] - ETA: 19s - loss: 74.6583 - acc: 0.0000e+00

 1080/17280 [>.............................] - ETA: 18s - loss: 66.9025 - acc: 0.0000e+00

...

10080/17280 [================>.............] - ETA: 10s - loss: 1.2905 - acc: 2.9762e-04

10440/17280 [=================>............] - ETA: 9s - loss: 1.2797 - acc: 2.8736e-04 

10800/17280 [=================>............] - ETA: 9s - loss: 1.3357 - acc: 2.7778e-04

11160/17280 [==================>...........] - ETA: 9s - loss: 1.3739 - acc: 2.6882e-04

11520/17280 [===================>..........] - ETA: 8s - loss: 1.3989 - acc: 2.6042e-04

In [12]:
model = from_file("simple-model.yaml")
model.compile(optimizer="rmsprop", loss="mae", metrics=["accuracy"])
model.summary()

hist = model.train((x_pca.reshape((x.shape[0], 1, -1, 2)), y), valid_data=(x_pca_test.reshape((x_test.shape[0], 1, -1, 2)), y_test), epochs=500)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
SkyFeatures (InputLayer)     (None, 1, 6208, 2)        0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 1, 6208, 1)        3         
_________________________________________________________________
Flatten1 (Flatten)           (None, 6208)              0         
_________________________________________________________________
Hidden1 (Dense)              (None, 360)               2235240   
_________________________________________________________________
Compass (Dense)              (None, 1)                 361       
Total params: 2,235,604
Trainable params: 2,235,604
Non-trainable params: 0
_________________________________________________________________
Train on 17280 samples, validate on 10440 samples


Epoch 1/500


  360/17280 [..............................] - ETA: 23s - loss: 1.3528 - acc: 0.0000e+00

  720/17280 [>.............................] - ETA: 19s - loss: 74.6583 - acc: 0.0000e+00

 1080/17280 [>.............................] - ETA: 18s - loss: 66.9025 - acc: 0.0000e+00

...

12960/17280 [=====================>........] - ETA: 6s - loss: 1.3700 - acc: 2.3148e-04

13320/17280 [======================>.......] - ETA: 6s - loss: 1.3579 - acc: 2.2523e-04

13680/17280 [======================>.......] - ETA: 5s - loss: 1.3436 - acc: 2.1930e-04

14040/17280 [=======================>......] - ETA: 5s - loss: 1.3399 - acc: 2.1368e-04

14400/17280 [========================>.....] - ETA: 4s - loss: 1.3252 - acc: 2.7778e-04